In [1]:
import torch

print("Test for GPU!")
print(torch.cuda.is_available())

Test for GPU!
True


# Load Dataset and Model

In [2]:
from datasets import load_dataset

# Load Dataset
dataset_name = "glue"
task_name = "sst2"
dataset = load_dataset(dataset_name, task_name, split="train")
validation = load_dataset(dataset_name, task_name, split="validation")

Found cached dataset glue (/home/9130/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Found cached dataset glue (/home/9130/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline

# Version 2-13b-chat
base_model_name = "meta-llama/Llama-2-13b-chat-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

device_map = {"": 0}

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map=device_map,
    trust_remote_code=True,
    use_auth_token=True
)

base_model.config.use_cache = False

# More info: https://github.com/huggingface/transformers/pull/24906
base_model.config.pretraining_tp = 1 

/usr/local/lib/python3.10/site-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer

LlamaTokenizerFast(name_or_path='meta-llama/Llama-2-13b-chat-hf', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False), 'pad_token': '</s>'}, clean_up_tokenization_spaces=False)

# Inference

## Create Prompt Template

### Few-shot Inference by Text-Generation

### Few-shot Inference by one-Chat

### Few-shot Inference by multi-Chat

In [77]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

# Define examples
examples = [
    {
        "instruction": "Classify the sentiment of the following text into 'positive' or 'negative':",
        "sentence": "I love this movie!",
        "label": "positive",
    },
    {
        "instruction": "Classify the sentiment of the following text into 'positive' or 'negative':",
        "sentence": "I am so bad today!",
        "label": "negative",
    },
]


In [80]:
# Define example prompt and few-shot prompt
example_prompt = PromptTemplate(
    input_variables=["instruction", "sentence", "label"], 
    template="<s><INST> Here is an inference example:\n{instruction}\nSentence: {sentence}\nSentiment: </INST> {label} </s>",
)

prompt = FewShotPromptTemplate(
    examples=examples, 
    example_prompt=example_prompt, 
    suffix="<s><INST> Classify the sentiment of the following text into 'positive' or 'negative':\nSentence: {sentence}\nSentiment: </INST>", 
    input_variables=["sentence"]
)

### Simple Test

In [81]:
sent2test = "I am so happy!"
prompt2test = prompt.format(sentence="I am so happy!")
print(prompt2test)

<s><INST> Here is an inference example:
Classify the sentiment of the following text into 'positive' or 'negative':
Sentence: I love this movie!
Sentiment: </INST> positive </s>

<s><INST> Here is an inference example:
Classify the sentiment of the following text into 'positive' or 'negative':
Sentence: I am so bad today!
Sentiment: </INST> negative </s>

<s><INST> Classify the sentiment of the following text into 'positive' or 'negative':
Sentence: I am so happy!
Sentiment: </INST>


In [68]:
prompt_manual = """\
<s><INST> Classify the sentiment of the following text into positive or negative:
Here is an inference example:
Sentence: I love this movie!
Sentiment: </INST> positive </s>

<s><INST> Here is an inference example:
Sentence: I am so bad today!
Sentiment: </INST> negative </s>

<s><INST> Classify the sentiment of the following text into positive or negative:
Sentence: I am so happy!
Sentiment: </INST>
"""

In [69]:
print(prompt_manual)

<s><INST> Classify the sentiment of the following text into positive or negative:
Here is an inference example:
Sentence: I love this movie!
Sentiment: </INST> positive </s>

<s><INST> Here is an inference example:
Sentence: I am so bad today!
Sentiment: </INST> negative </s>

<s><INST> Classify the sentiment of the following text into positive or negative:
Sentence: I am so happy!
Sentiment: </INST>



In [82]:
inputs = tokenizer(prompt2test, return_tensors="pt").to("cuda")
outputs = base_model.generate(input_ids=inputs["input_ids"].to("cuda"), attention_mask=inputs["attention_mask"], max_new_tokens=80, pad_token_id=tokenizer.eos_token_id)

print(tokenizer.decode(outputs[0], skip_special_tokens=False))

<s><s> <INST> Here is an inference example:
Classify the sentiment of the following text into 'positive' or 'negative':
Sentence: I love this movie!
Sentiment: </INST> positive </s> 

<s> <INST> Here is an inference example:
Classify the sentiment of the following text into 'positive' or 'negative':
Sentence: I am so bad today!
Sentiment: </INST> negative </s> 

<s> <INST> Classify the sentiment of the following text into 'positive' or 'negative':
Sentence: I am so happy!
Sentiment: </INST> positive</s>


### Inference and evaluate

In [91]:
import torch
from torch.utils.data import Dataset, DataLoader

class ChatDataset(Dataset):
    def __init__(self, dataset, tokenizer, label_map, prompt):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.label_map = label_map
        self.prompt = prompt

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        val = self.dataset[idx]
        label_text = self.label_map[val['label']]
        sentence = val['sentence'][:-1]
        text = self.prompt.format(sentence=sentence)
        inputs = self.tokenizer(text, return_tensors="pt").to("cuda")
        return inputs, label_text


In [138]:
def collate_fn(batch):
    # Split batch into inputs and labels
    input_ids, labels = zip(*batch)

    # Compute the max length
    # print("Input_ids: ", input_ids)
    # print("Len(input_ids): ", len(input_ids))
    max_length = max(ids["input_ids"].shape[0] for ids in input_ids)

    # Pad the sequences
    inputs = tokenizer.pad([ids["input_ids"].tolist() for ids in input_ids], 
                           padding='max_length', 
                           max_length=max_length,
                           return_tensors="pt")

    return inputs, labels


In [139]:
import re
from tqdm import tqdm

def evaluate_chat(dataset, prompt):
    label_map = {
        0 : 'negative',
        1 : 'positive',
    }

    batch_size = 16
    dataset = ChatDataset(dataset, tokenizer, label_map, prompt)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

    compared_result = []
    invalid_label = []

    for i, (inputs, labels) in enumerate(tqdm(data_loader)):
        # label_text = label_map[val['label']]
        # sentence = val['sentence'][:-1]

        # Make input
        # text = prompt.format(sentence=sentence)
        # inputs = tokenizer(text, return_tensors="pt").to("cuda")

        # Generate
        outputs = base_model.generate(input_ids=inputs["input_ids"].to("cuda"), attention_mask=inputs["attention_mask"], max_new_tokens=80, pad_token_id=tokenizer.eos_token_id)
        # print(tokenizer.decode(outputs[0], skip_special_tokens=True))
        outputs_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # print(outputs_text)
        selected_sentiment = outputs_text.split("\n")[-1].lower()
        # selected_sentiment = remove_last_punctuation(selected_sentiment)
        selected_sentiment = selected_sentiment.split(" ")[-1]
        # print(selected_sentiment)
        # if i == 1:
        #     break

        # Abnormal case
        # if selected_sentiment not in ['positive', 'negative']:
        #     # invalid_index.append(i)
        #     invalid_label.append(selected_sentiment)
        #     compared_result.append(0)
        #     continue

        # Compare prediction and label
        # assert selected_sentiment in ['positive', 'negative'], f"Prediction {i} is not valid: {selected_sentiment}"
        for idx in len(batch_size):
            if selected_sentiment[idx] not in ['positive', 'negative']:
                invalid_label.append(selected_sentiment[idx])
                compared_result.append(0)
                continue
            
            if selected_sentiment[idx] == labels[idx]:
                compared_result.append(1)
            else:
                compared_result.append(0)

    return compared_result, invalid_label


In [140]:
comp_res, invalid_label = evaluate_chat(validation, prompt)

  0%|          | 0/55 [00:00<?, ?it/s]


AttributeError: 'list' object has no attribute 'keys'

In [ ]:
print("Accuracy:", comp_res.count(1)/len(comp_res))

In [87]:
print("Accuracy:", comp_res.count(1)/len(comp_res))

Accuracy: 0.8314220183486238


In [88]:
len(invalid_label)

115

In [90]:
from collections import Counter

counted_elements = Counter(invalid_label)

print(counted_elements)


Counter({'neutral': 108, 'mixed': 7})
